Simple product state

In [1]:
using ITensors
using ITensorMPS

# Define the Hadamard gate
function hadamard(s)
    sx = op("Sx", s)
    sz = op("Sz", s)

    h = (sx + sz) * sqrt(2.0)
    
    return h
end


N = 5 #number of qubits
sites = siteinds("S=1/2", N)
psi0 = productMPS(sites, "Up") #all in the up-state (|0>)


ampo = AutoMPO()
for j in 1:N
  ampo += 2.0, "Sx", j                  # add an Sx on site j
end
mpoX = MPO(ampo, sites)            # convert your sum into an MPO
val = inner(psi0', mpoX, psi0)    # contracts bra ψ, mpoX, ket ψ
println("⟨X₁ + X₂ + ⋯ + X_N⟩ = ", real(val))

product_tensor = ITensor(1)
for j in 1:N
    s = sites[j]
    product_tensor *= 2 * op("Sx", s)
end
# Create an MPO from the product tensor
mpoSigmaX = MPO(product_tensor, sites)
val = inner(psi0', mpoSigmaX, psi0)
println("⟨X₁ * X₂ * ⋯ * X_N⟩ = ", real(val))


⟨X₁ + X₂ + ⋯ + X_N⟩ = 0.0
⟨X₁ * X₂ * ⋯ * X_N⟩ = 0.0


In [2]:
# Apply Hadamard to site 3
site_index = 3
h = hadamard(sites[site_index])

# Apply the gate to the MPS - use an array for the site index
psi = apply(h, psi0, [site_index])  # Note the [site_index] array syntax here

# Normalize the state
psi = normalize(psi)

# Verify by computing expectation values
for j in 1:N
    # Create a simple MPO for each measurement
    ampo_x = AutoMPO()
    ampo_x += 2.0, "Sx", j
    mpo_x = MPO(ampo_x, sites)
    
    ampo_z = AutoMPO()
    ampo_z += 2.0, "Sz", j
    mpo_z = MPO(ampo_z, sites)
    
    expval_x = real(inner(psi', mpo_x, psi))
    expval_z = real(inner(psi', mpo_z, psi))
    
    println("Site $j: ⟨σx⟩ = $(round(expval_x, digits=4)), ⟨σz⟩ = $(round(expval_z, digits=4))")
end

println("⟨X₁ + X₂ + ⋯ + X_N⟩ = $(round(real(inner(psi', mpoX, psi)), digits=4))")
println("⟨X₁ * X₂ * ⋯ * X_N⟩ = $(round(real(inner(psi', mpoSigmaX, psi)), digits=4))")


Site 1: ⟨σx⟩ = 0.0, ⟨σz⟩ = 1.0
Site 2: ⟨σx⟩ = 0.0, ⟨σz⟩ = 1.0
Site 3: ⟨σx⟩ = 1.0, ⟨σz⟩ = 0.0
Site 4: ⟨σx⟩ = 0.0, ⟨σz⟩ = 1.0
Site 5: ⟨σx⟩ = 0.0, ⟨σz⟩ = 1.0
⟨X₁ + X₂ + ⋯ + X_N⟩ = 1.0
⟨X₁ * X₂ * ⋯ * X_N⟩ = 0.0


In [3]:
sites_to_apply = [1, 2, 3, 4, 5]
psi = psi0
for site_index in sites_to_apply
    h = hadamard(sites[site_index])
    psi = apply(h, psi, [site_index])  # Note the brackets
    psi = normalize(psi)
end

# Verify by computing expectation values
for j in 1:N
    # Create a simple MPO for each measurement
    ampo_x = AutoMPO()
    ampo_x += 2.0, "Sx", j
    mpo_x = MPO(ampo_x, sites)
    
    ampo_z = AutoMPO()
    ampo_z += 2.0, "Sz", j
    mpo_z = MPO(ampo_z, sites)
    
    expval_x = real(inner(psi', mpo_x, psi))
    expval_z = real(inner(psi', mpo_z, psi))
    
    println("Site $j: ⟨σx⟩ = $(round(expval_x, digits=4)), ⟨σz⟩ = $(round(expval_z, digits=4))")
end

println("⟨X₁ + X₂ + ⋯ + X_N⟩ = $(round(real(inner(psi', mpoX, psi)), digits=4))")
println("⟨X₁ * X₂ * ⋯ * X_N⟩ = $(round(real(inner(psi', mpoSigmaX, psi)), digits=4))")

Site 1: ⟨σx⟩ = 1.0, ⟨σz⟩ = -0.0
Site 2: ⟨σx⟩ = 1.0, ⟨σz⟩ = 0.0
Site 3: ⟨σx⟩ = 1.0, ⟨σz⟩ = 0.0
Site 4: ⟨σx⟩ = 1.0, ⟨σz⟩ = -0.0
Site 5: ⟨σx⟩ = 1.0, ⟨σz⟩ = 0.0
⟨X₁ + X₂ + ⋯ + X_N⟩ = 5.0
⟨X₁ * X₂ * ⋯ * X_N⟩ = 1.0


Two-qubit unitary

In [4]:
using ITensors
using ITensorMPS
using Optim
using LinearAlgebra
using Printf
using Random

# Define basic operators
function pauli_operators(s)
    sigmax = 2 * op("Sx", s)
    sigmay = 2 * op("Sy", s)
    sigmaz = 2 * op("Sz", s)
    id = op("Id", s)
    return [id, sigmax, sigmay, sigmaz]  # σ⁰, σˣ, σʸ, σᶻ
end

# Define the Hadamard gate
function hadamard(s)
    sx = op("Sx", s)
    sz = op("Sz", s)
    h = (sx + sz) * sqrt(2.0)
    return h
end

N = 5 #number of qubits
sites = siteinds("S=1/2", N)
psi0 = productMPS(sites, "Up") #all in the up-state (|0>)
#act with Hadamard on the first site
site_index = 1
h = hadamard(sites[site_index])
# Apply the gate to the MPS - use an array for the site index
psi0 = apply(h, psi0, [site_index])  # Note the [site_index] array syntax here
psi0 = normalize(psi0);

# Create parameterized two-qubit unitary, acting on sites i and i + 1
function create_unitary(sites, i, params)
    # params has 15 parameters that encode the unitary
    
    s1 = sites[i]
    s2 = sites[i+1]
    
    # Get Pauli operators for both sites
    ops1 = pauli_operators(s1)  # [I, σx, σy, σz] for site 1
    ops2 = pauli_operators(s2)  # [I, σx, σy, σz] for site 2
    
    # Initialize the generator of the unitary (Hermitian operator)
    generator = 0.0 * ops1[1] * ops2[1]#pick custom phase 
    
    # Two-qubit σᵃ⊗σᵇ terms
    param_idx = 1
    for a in 2:4, b in 2:4
        term = ops1[a] * ops2[b]
        generator += -im * params[param_idx] * term
        param_idx += 1
    end

    #Single-qubit σᵃ⊗I terms
    for a in 2:4
        term = ops1[a] * ops2[1]
        generator += -im * params[param_idx] * term
        param_idx += 1
    end

    #Single-qubit I⊗σᵇ terms
    for b in 2:4
        term = ops1[1] * ops2[b]
        generator += -im * params[param_idx] * term
        param_idx += 1
    end

    return exp(generator)
end

# Apply the ladder circuit to a given MPS psi
function apply_circuit(psi, sites, params)
    N = length(sites)
    psi_evolved = deepcopy(psi)
    # Apply unitaries sequentially: (1,2), (2,3), ...
    for i in 1:(N-1)
        U = create_unitary(sites, i, params)
        psi_evolved = apply(U, psi_evolved, [i, i+1])
        normalize!(psi_evolved)
    end   
    return psi_evolved
end

function cost_function(psi, sites, g)
    N = length(sites)

    X_string = ITensor(1)
    for j in 1:N
        s = sites[j]
        X_string *= 2 * op("Sx", s)
    end
    X_string = MPO(X_string, sites)

    O_string = ITensor(1)
    O_string *= 2 * op("Sz", sites[1])
    O_string *= 2 * op("Sy", sites[2])
    for j in 3:(N - 2)
        s = sites[j]
        O_string *= 2 * op("Sx", s)
    end
    O_string *= 2 * op("Sy", sites[N - 1])
    O_string *= 2 * op("Sz", sites[N])
    O_string = MPO(O_string, sites)


    ampo = AutoMPO()
    # Add Z_j * Z_{j+1} terms for each neighboring pair
    for j in 1:(N-1)
        ampo .+= 4.0, "Sz", j, "Sz", j + 1
    end
    ZZ_term = MPO(ampo, sites)

    return -real(inner(psi', X_string, psi)) - 0.5 * (1 - g) * real(inner(psi', O_string, psi)) - 0.5 * (1 + g) * real(inner(psi', ZZ_term, psi)) / (N - 1)
end

function cost_function_params(params, psi0, sites, g)
    psi = apply_circuit(psi0, sites, params)
    return cost_function(psi, sites, g)
end

# Function to optimize the parameters
function optimize_circuit(psi0, sites, g, initial_params; max_iter=1000)
    N = length(sites)
    
    # Create the objective function for optimization
    function objective(p)
        cost = cost_function_params(p, psi0, sites, g)
        # Print progress
        # if rand() < 0.1  # Only print occasionally to avoid too much output
        #     println("Current cost: $cost")
        # end
        return cost
    end
    
    # Run the optimization
    println("Starting optimization...")
    result = optimize(
        objective,
        initial_params,
        BFGS(),
        Optim.Options(iterations=max_iter, show_trace=true)
    )
    
    # Get optimized parameters
    opt_params = Optim.minimizer(result)
    
    # Apply optimized parameters to get final state
    final_psi = apply_circuit(psi0, sites, opt_params)
    
    final_cost = cost_function(final_psi, sites, g)
    
    println("Optimization complete!")
    println("Final cost: $final_cost")

    return opt_params
end

params = 0.01 * randn(15)
params_GHZ = zeros(15)
params_GHZ[7] = pi / 4
params_GHZ[12] = - pi / 4
params_GHZ[13] = - pi / 4

params_plus = zeros(15)
params_plus[14] = pi / 4

# params = params_GHZ
g = -1

final_params = optimize_circuit(psi0, sites, g, params)


Starting optimization...
Iter     Function value   Gradient norm 
     0     3.121069e-04     1.219261e-01
 * time: 0.016774892807006836
     1    -1.302679e-03     1.311738e-01
 * time: 0.7534739971160889
     2    -3.133052e-01     1.295313e+00
 * time: 1.0001699924468994
     3    -3.170468e-01     1.181304e+00
 * time: 1.376446008682251
     4    -3.253529e-01     8.732262e-01
 * time: 1.5432548522949219
     5    -3.410044e-01     6.663185e-01
 * time: 1.778601884841919
     6    -5.010320e-01     6.873465e-01
 * time: 1.9479238986968994
     7    -7.414187e-01     8.666180e-01
 * time: 2.1109700202941895
     8    -8.017977e-01     7.923808e-01
 * time: 2.4866859912872314
     9    -8.698041e-01     8.540210e-01
 * time: 2.705206871032715
    10    -9.254094e-01     6.713169e-01
 * time: 2.9445807933807373
    11    -9.877698e-01     6.754906e-01
 * time: 3.109041929244995
    12    -1.000217e+00     6.005022e-01
 * time: 3.282528877258301
    13    -1.029560e+00     2.458269e-01

15-element Vector{Float64}:
 -0.7384932133947707
 -0.14783858122074925
 -0.0012894233121173429
 -0.07297674523260456
 -0.4085709733111911
  1.0243962302153642
 -0.48675435689911983
 -0.4425407664012404
 -0.5831437330085708
 -0.0626565295127135
 -0.026026244678916968
 -0.1072657224094252
 -0.06444123449934074
 -0.41543562120901445
  0.5835413264799251

Fidelity with respect to (1) GHZ state, (2) cluster state, and (3) product state |+...+>

In [5]:

psi = apply_circuit(psi0, sites, final_params)

# Create |00...0> state
state_0 = productMPS(sites, "Up")
# Create |11...1> state
state_1 = productMPS(sites, "Dn")
# Create the superposition: (|00...0> + |11...1>)/sqrt(2)
ghz_state = add(state_0, state_1)
normalize!(ghz_state)

fidelity_ghz = abs2(inner(psi, ghz_state))
println("Fidelity wrt to GHZ = $(round(fidelity_ghz, digits=4))")

# Calculate fidelity with cluster state
# First create |+>⊗|+>...⊗|+> state
cluster_state = MPS(sites, n -> "X+")
# Apply CZ gates between neighboring qubits
for j in 1:(length(sites)-1)
    # Create CZ gate between sites j and j+1
    s1 = sites[j]
    s2 = sites[j+1]
    
    # CZ = |00><00| + |01><01| + |10><10| - |11><11|
    # In computational basis, this corresponds to a phase flip when both qubits are in |1> state
    proj00 = op("ProjUp", s1) * op("ProjUp", s2)
    proj01 = op("ProjUp", s1) * op("ProjDn", s2)
    proj10 = op("ProjDn", s1) * op("ProjUp", s2)
    proj11 = op("ProjDn", s1) * op("ProjDn", s2)
    
    cz = proj00 + proj01 + proj10 - proj11
    
    # Apply the gate
    cluster_state = apply(cz, cluster_state, [j, j+1])
    normalize!(cluster_state)
end

# Calculate fidelity with cluster state
fidelity_cluster = abs2(inner(psi, cluster_state))
println("Fidelity wrt to cluster_state = $(round(fidelity_cluster, digits=4))")


# Create |+>⊗|+>...⊗|+> state
plus_state = MPS(sites, n -> "X+")
fidelity_plus = abs2(inner(psi, plus_state))
println("Fidelity wrt to |+>⊗|+>...⊗|+> = $(round(fidelity_plus, digits=4))")


Fidelity wrt to GHZ = 0.1185
Fidelity wrt to cluster_state = 0.0392
Fidelity wrt to |+>⊗|+>...⊗|+> = 0.0006


In [12]:
# Function to optimize the parameters
function optimize_circuit_state(psi0, sites, g, initial_params, state; max_iter=1000)
    N = length(sites)
    
    # Create the objective function for optimization
    function objective(p)
        psi = apply_circuit(psi0, sites, p)
        cost = -abs2(inner(psi, state))
        # Print progress
        if rand() < 0.1  # Only print occasionally to avoid too much output
            println("Current cost: $cost")
        end
        return cost
    end
    
    # Run the optimization
    println("Starting optimization...")
    result = optimize(
        objective,
        initial_params,
        BFGS(),
        Optim.Options(iterations=max_iter, show_trace=true)
    )
    
    # Get optimized parameters
    opt_params = Optim.minimizer(result)
   
    println("Optimization complete!")
    println("Final cost: $objective(opt_params)")

    return opt_params
end


final_params = optimize_circuit_state(psi0, sites, g, params, cluster_state)
psi = apply_circuit(psi0, sites, final_params)
# Calculate fidelity with cluster state
fidelity_cluster = abs2(inner(psi, cluster_state))
println("Fidelity wrt to cluster_state = $(round(fidelity_cluster, digits=4))")

params_cluster = final_params

Starting optimization...
Current cost: -0.06112652996770745
Current cost: -0.06112666804899048
Current cost: -0.06112426605749725
Iter     Function value   Gradient norm 
     0    -6.112649e-02     4.849169e-01
 * time: 7.82012939453125e-5
Current cost: -0.0048265382653689725
Current cost: -0.004826338236769936
Current cost: -0.004826001869979073
Current cost: -0.25240528689683095
Current cost: -0.2524109845892194
Current cost: -0.2524112224752585
Current cost: -0.26589264954039943
Current cost: -0.2658900095672698
Current cost: -0.2658953957789333
     1    -2.658953e-01     8.673121e-01
 * time: 0.09931612014770508
Current cost: -0.0035023636807742205
Current cost: -0.0035020902856279434
Current cost: -0.003502372594085058
Current cost: -0.0035021871501655145
Current cost: -0.003500766669206599
Current cost: -0.04925882259456753
Current cost: -0.04925932011220381
Current cost: -0.0492593286127466
Current cost: -0.030310182996516555
Current cost: -0.5403154446655873
Current cost: -0.

15-element Vector{Float64}:
  0.09306157226169659
 -0.004331368734483807
 -1.0021610271716632
 -2.864660677811201
  0.13332953033649872
  6.609962394650361
  0.4742350798276221
 10.189208493826445
  0.2776344908173366
  0.13332954216418583
  0.0043313713700696655
  0.37815042532647497
  0.2776344803679213
  5.965133538895049
  0.4742350753889814

In [13]:
params = params_cluster

g = -1

final_params = optimize_circuit(psi0, sites, g, params)

Starting optimization...
Iter     Function value   Gradient norm 
     0     1.000000e+00     7.053601e-09
 * time: 4.887580871582031e-5
Optimization complete!
Final cost: 0.9999999999999999


15-element Vector{Float64}:
  0.09306157226169659
 -0.004331368734483807
 -1.0021610271716632
 -2.864660677811201
  0.13332953033649872
  6.609962394650361
  0.4742350798276221
 10.189208493826445
  0.2776344908173366
  0.13332954216418583
  0.0043313713700696655
  0.37815042532647497
  0.2776344803679213
  5.965133538895049
  0.4742350753889814